In [ ]:
import pandas as pd
import numpy as np

## Data Load

In [ ]:
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
PATH = '/kaggle/working/'
train = pd.read_csv(PATH+'labeledTrainData.tsv', delimiter='\t', quoting=3)
test = pd.read_csv(PATH+'testData.tsv', delimiter='\t', quoting=3)
unlabeled_train = pd.read_csv(PATH+'unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

In [ ]:
print(train.shape)
print(test.shape)
print(unlabeled_train.shape)

print(train['review'].size)
print(test['review'].size)
print(unlabeled_train['review'].size)

In [ ]:
!wget https://github.com/Bligh-Park/Kaggle/raw/main/Word2VecUtil.py

In [ ]:
from Word2VecUtil import KaggleWord2VecUtility

In [ ]:
KaggleWord2VecUtility.review_to_wordlist(train['review'][0][:50])

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
sentences = []
for review in train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(
        review, remove_stopwords=False)

In [ ]:
for review in unlabeled_train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(
        review, remove_stopwords=False)

In [ ]:
len(sentences)

In [ ]:
sentences[0][:10]

In [ ]:
sentences[1][:10]

## Word2Vec 모델을 학습
전처리를 거쳐 파싱된 문장의 목록으로 모델을 학습시킬 준비가 됨
### word2vec 모델의 파라미터
* 아키텍처 : 아키텍처 옵션은 skip-gram(default) 또는 CBOW모델. skip-gram은 느리지만 더 나은 결과가 나옴
* 학습 알고리즘 : Hierachical softmax(default) 또는 negative 샘플링. 여기서는 기본값이 잘 동작.
* 빈번하게 등장하는 단어에 대한 다운 샘플링 : Google 문서는 0.00001에서 0.001 사이의 값을 권장한다. 여기에서는 0.001에 가까운 값이 최종 모델의 정확도를 높이는 것으로 보여진다.
* 단어벡터 차원 : 많은 feature를 사용한다고 항상 좋은 것은 아니지만 대체적으로 좀 더 나은 모델이 된다. 합리적인 값은 수십에서 수백 개가 될 수 있고 여기에서는 300으로 지정.
* 컨텍스트/창 크기 : 학습 알고리즘이 고려해야 하는 컨텍스트의 단어 수는 얼마나 될까? hierachical softmax를 위해 좀 더 큰 수가 좋지만 10 정도가 적당
* worker threads : 실행 할 병렬 프로세스의 수로 컴퓨터마다 다르지만 대부분의 시스템에서 4에서 6 사이의 값을 사용
* 최소 단어 수 : 어휘의 크기를 의미하는 단어로 제한하는 데 도움이 된다. 모든 문서에서 여러번 발생하지 않는 단어는 무시된다. 10에서 100사이가 적당하며 이 경진대회의 데이터는 각 영화가 30개씩 리뷰가 있기 때문에 개별 영화 제목에 너무 많은 중요성이 붙는 것을 피하기 위해 최소 단어 수를 40으로 설정한다. 그 결과 전체 어휘 크기는 약 15,000 단어가 된다. 높은 값은 제한 된 실행시간에 도움이 된다.

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

In [ ]:
# 파라미터 값 지정
num_features = 300 # 문자 벡터 차원 수
min_word_count = 40 # 최소 무자 수
num_workers = 4 # 병렬 처리 쓰레드 수
context = 10 # 문자열 창 크기
downsampling = 1e-3 # 문자 빈도 수 downsample

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(sentences,
                         workers=num_workers,
                         size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling)
model

In [ ]:
model

In [ ]:
# 학습 완료 후 필요없는 메모리 unload
model.init_sims(replace=True)

model_name = '300features_40minwords_10text'

model.save(model_name)

## 모델 결과 탐색

In [ ]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('man woman child kitchen'.split())

In [ ]:
model.wv.doesnt_match('france england germany berlin'.split())

In [ ]:
# 가장 유사한 단어를 추출
model.wv.most_similar('man')

In [ ]:
model.wv.most_similar('queen')

In [ ]:
# vocab 에 없는 단어 테스트
model.wv.most_similar('awful')

In [ ]:
model.wv.most_similar('film')

In [ ]:
model.wv.most_similar('happy')

In [ ]:
# stemming 되어 있는 것 고려
model.wv.most_similar('happi')

## Word2Vec으로 벡터화한 단어를 t-SNE를 통해 시각화

In [ ]:
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import gensim
import gensim.models as g

In [ ]:
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
model_name = '300features_40minwords_10text'
model = g.Doc2Vec.load(model_name)

vocab = list(model.wv.vocab)
X = model[vocab]

print(len(X))
print(X[0][:10])
tsne = TSNE(n_components=2)

# 100개의 단어에 대해서만 시각화
X_tsne = tsne.fit_transform(X[:100, :])

In [ ]:
df = pd.DataFrame(X_tsne, index=vocab[:100], columns=['x', 'y'])
df.shape

In [ ]:
df.head(10)

In [ ]:
fig = plt.figure()
fig.set_size_inches(40, 20)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'])

for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=30)
plt.show()

In [ ]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    '''
    주어진 문장에서 단어 벡터의 평균을 구하는 함수
    '''
    # 속도를 위해 0으로 채운 배열을 초기화
    featureVec = np.zeros((num_features,), dtype='float32')
    
    nwords = 0.
    # Index2word는 모델의 사전에 있는 단어명을 담은 리스트이다.
    # 속도를 위해 set 형태로 초기화 한다.
    index2word_set = set(model.wv.index2word)
    
    # 루프를 돌며 모델 사전에 포함이 되는 단어라면 피처에 추가한다.
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    
    # 결과를 단어수로 나누어 평균을 구한다.
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [ ]:
def getAvgFeatureVecs(reviews, model, num_features):
    # 리뷰 단어 목록의 각각에 대한 평균 feature 벡터를 계산하고
    # 2D numpy 배열을 반환한다.
    
    # 카운터를 초기화
    counter = 0.
    # 속도를 내기 위해 2d 넘파이 배열을 미리 할당
    reviewFeatureVecs = np.zeros(
        (len(reviews), num_features), dtype='float32')
    
    for review in reviews:
        # 매 1000개 리뷰마다 상태를 출력
        if counter%1000. == 0.:
            print('Review %d of %d' % (counter, len(reviews)))
        # 평균 피처 벡트를 만들기 우해 위에서 정의한 함수를 호출
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
                                                        num_features)
        # 카운터를 증가
        counter = counter + 1.
    return reviewFeatureVecs

In [ ]:
# 멀티스레드로 4개의 워커를 사용해 처리한다.
def getCleanReviews(reviews):
    clean_reviews = []
    clean_reviews = KaggleWord2VecUtility.apply_by_multiprocessing(\
            reviews['review'], KaggleWord2VecUtility.review_to_wordlist,\
            workers=4)
    return clean_reviews

In [ ]:
%time
trainDataVecs = getAvgFeatureVecs(\
    getCleanReviews(train), model, num_features)

In [ ]:
%time
testDataVecs = getAvgFeatureVecs(\
    getCleanReviews(test), model, num_features)

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(
    n_estimators=100, n_jobs=-1, random_state=2018)

In [ ]:
%time
forest = forest.fit(trainDataVecs, train['sentiment'])

In [ ]:
from sklearn.model_selection import cross_val_score
%time
score = np.mean(cross_val_score(\
        forest, trainDataVecs, \
        train['sentiment'], cv=10, scoring='roc_auc'))

In [ ]:
score

In [ ]:
result = forest.predict(testDataVecs)

## 캐글 제출 파일 생성

In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.to_csv('Word2Vec_AverageVEctors_{0:.5f}.csv'.format(score),
             index=False, quoting=3)

## 결과 확인

In [ ]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

In [ ]:
import seaborn as sns
%matplotlib inline

fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.countplot(train['sentiment'], ax=axes[0])
sns.countplot(output['sentiment'], ax=axes[1])